Data preparing

In [67]:
import pandas as pd
import pickle

In [71]:
import twitter

CONSUMER_KEY = "eTG832aIi0pyU6EjfoTGge6gx"
CONSUMER_SECRET = "mjvQFaovxMuRJ0U5XpdYfPsmmqiaZFNsL7g4qkk942OdDip6hG"

ACCESS_TOKEN_KEY = "780018504787488769-KVrRy17YTgqcUzrM32ykuYipvptNKvB"
ACCESS_TOKEN_SECRET = "qtZmbBAiOrzaBxwGNdlP0bfVA3cz2AWmsv0kq1jfdIOEb"

api = twitter.Api(consumer_key=CONSUMER_KEY, 
                  consumer_secret=CONSUMER_SECRET, 
                  access_token_key=ACCESS_TOKEN_KEY, 
                  access_token_secret=ACCESS_TOKEN_SECRET,
                  sleep_on_rate_limit=True)

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test_submition.csv")
test.cls = None

In [100]:
def get_user_twts (u_id):
    kaggle_date = 1474243200
    timeline = []
    timeline = api.GetUserTimeline(user_id=u_id, count=200, include_rts=False, exclude_replies=True)
    for i in xrange(20):
        if len(timeline) == 0: return []
        oldest_twt_id = timeline.pop().id
        timeline += api.GetUserTimeline(user_id=u_id, max_id=oldest_twt_id, count=200, include_rts=False, exclude_replies=True) 
        if timeline[-1].created_at_in_seconds < kaggle_date:
            print "timeline get iterations:", i
            break
    oldest_twt_id = timeline.pop().id
    timeline += api.GetUserTimeline(user_id=u_id, max_id=oldest_twt_id, count=200, include_rts=False, exclude_replies=True) 
    if timeline[-1].created_at_in_seconds > kaggle_date: 
        print "BAD_USER"
        bad_users.add(u_id)
    return timeline


In [6]:
import unicodedata
import nltk
import re

wnl = nltk.stem.wordnet.WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')
tokenizer = nltk.RegexpTokenizer('[a-z]+')

def tokenize_twt(twt):
    text = twt.text
    #normalization
    text = ''.join(c for c in unicodedata.normalize('NFD', text.lower()) if not unicodedata.combining(c))
    #deleting URL's
    text = re.sub('https?://\S+', '', text)
    #deleting HTML symbols
    text = re.sub('&\w+;', '', text)    

    words = tokenizer.tokenize(text)
    tokens = [wnl.lemmatize(word) for word in words if word not in stopwords]
    return tokens

def get_user_vocab(twts):
    vocab = {}
    for twt in twts:
        tokens = tokenize_twt(twt)
        for token in tokens:
            if token in vocab:
                vocab[token] += 1
            else:
                vocab[token] = 1
    return vocab


In [29]:
user_tokens = {}

In [30]:
bad_users = set()

In [31]:
downloaded_users = set()

In [101]:
from twitter import TwitterError
from tqdm import tqdm
user_ids = pd.concat([train, test])['uid']
for uid in tqdm(user_ids):
    if uid in downloaded_users: continue
    if len(user_tokens) >= 1000:
        print 'TRYING TO SAVE PICKLE'
        with open('user_tokens_'+str(len(downloaded_users))+'.pkl', 'wb') as f:
            pickle.dump(user_tokens, f, pickle.HIGHEST_PROTOCOL)
            print "pickle dumped", 'user_tokens_'+str(len(downloaded_users))+'.pkl'
            user_tokens = {}
    try:    
        twts = get_user_twts(uid)
    except TwitterError:
        twts = []
        print 'error'
        next
    vocab = get_user_vocab(twts)
    user_tokens[uid] = vocab
    downloaded_users.add(uid)
    print uid, len(twts), 'downloaded', len(vocab), 'words in vocab'
        
    

 12%|█▏        | 1464/11947 [00:18<02:10, 80.60it/s]

BAD_USER
63517561 642 downloaded 1585 words in vocab
BAD_USER
65552940 673 downloaded 2327 words in vocab
timeline get iterations: 1
1031499828 204 downloaded 825 words in vocab
timeline get iterations: 0
2606851334 587 downloaded 2750 words in vocab
timeline get iterations: 0
219549026 51 downloaded 270 words in vocab
timeline get iterations: 0
63100957 598 downloaded 2209 words in vocab
timeline get iterations: 9
16860135 577 downloaded 2232 words in vocab
timeline get iterations: 12
152987478 2986 downloaded 3772 words in vocab
timeline get iterations: 1
59085872 466 downloaded 1800 words in vocab
timeline get iterations: 12
193487520 221 downloaded 329 words in vocab
timeline get iterations: 10
742994089 759 downloaded 2105 words in vocab
timeline get iterations: 0
2395121324 178 downloaded 363 words in vocab
BAD_USER
1377836916 1082 downloaded 1787 words in vocab
timeline get iterations: 5
19991010 793 downloaded 2143 words in vocab
timeline get iterations: 1
138134164 320 downloa

IOError: [Errno 22] Invalid argument

In [69]:
reload (twitter)

<module 'twitter' from '/home/dima/anaconda2/lib/python2.7/site-packages/twitter/__init__.pyc'>

In [102]:
len(user_tokens)

629

In [103]:
len(downloaded_users)

1629

In [109]:
len(bad_users)

338

In [105]:
import time
def save_downloaded_users_set():
    t = int(time.time())
    with open(str(t)+'_downloaded_users.pkl', 'wb') as f:
        pickle.dump(downloaded_users, f, pickle.HIGHEST_PROTOCOL)
        print "pickle dumped", str(t)+'_downloaded_users.pkl'
    return

def save_bad_users_set():
    t = int(time.time())
    with open(str(t)+'_bad_users.pkl', 'wb') as f:
        pickle.dump(bad_users, f, pickle.HIGHEST_PROTOCOL)
        print "pickle dumped", str(t)+'_bad_users.pkl'
    return

def save_user_tokens():
    t = int(time.time())
    with open(str(t)+'_user_tokens.pkl', 'wb') as f:
        pickle.dump(user_tokens, f, pickle.HIGHEST_PROTOCOL)
        print "pickle dumped", str(t)+'_user_tokens.pkl'
    return

In [106]:
save_downloaded_users_set()
save_bad_users_set()
save_user_tokens()

pickle dumped 1482080100_downloaded_users.pkl
pickle dumped 1482080101_bad_users.pkl
pickle dumped 1482080101_user_tokens.pkl


In [107]:
def load_pickle(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [110]:
print 'load test'

print 'downloaded', len(load_pickle('1482080100_downloaded_users.pkl'))
print 'bad', len(load_pickle('1482080101_bad_users.pkl'))
print 'tokens', len(load_pickle('1482080101_user_tokens.pkl'))

load test
downloaded 1629
bad 338
tokens 629


In [96]:
!ls

bad_users1482077419.43.pkl	   test_submition.csv
bad_users1482077865.63.pkl	   train.csv
downloaded_users1482077419.34.pkl  user_tokens1482077419.45.pkl
downloaded_users1482077865.54.pkl  user_tokens_1482077865.64.pkl
Project. Data Preraring.ipynb	   user_tokens_1.pkl
test.csv
